[View in Colaboratory](https://colab.research.google.com/github/JozeeLin/google-tensorflow-exercise/blob/master/ch03_tensorflow_%E7%AC%AC%E4%B8%80%E6%AD%A5.ipynb)

In [0]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
#直接加载数据集
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py fr

In [5]:
#训练集55000个样本，验证集5000个样本，测试集10000个样本
#由于图像是28像素x28像素大小的灰度图片，因此每个样本有28x28=784维特征
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)
print(mnist.validation.images.shape, mnist.validation.labels.shape)

(55000, 784) (55000, 10)
(10000, 784) (10000, 10)
(5000, 784) (5000, 10)


In [0]:
#创建一个新的InteractiveSession，使用这个命令会将这个session注册为默认的session，之后的运算也默认跑在这个session里，不同session之间的数据和运算应该都是相互独立的
sess = tf.InteractiveSession()

In [0]:
#创建一个Placeholder，即输入数据的地方，Placeholder第一个参数是数据类型，第二个参数代表tensor的shape，也就是数据的尺寸。
x = tf.placeholder(tf.float32, [None,784])

In [0]:
#给softmax regression 模型中的weights和biases创建Variable对象
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

In [0]:
#实现softmax regression 算法
y = tf.nn.softmax(tf.matmul(x,W)+b)

softmax是tf.nn下面的一个函数，而tf.nn则包含了大量神经网络的组件，tf.matmul是TensorFlow中的矩阵乘法函数。

In [0]:
#定义损失函数loss function来描述模型对问题的分类精度。Loss越小，代表模型的分类结果与真实值得偏差越小，也就是说模型越精确。对多分类问题，通常使用cross-entropy作为损失函数。
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y), reduction_indices=[1]))

In [0]:
#选择优化算法
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [0]:
#使用全局参数初始化器，并直接执行它的run方法
tf.global_variables_initializer().run()

In [0]:
#最后一步，迭代地执行训练操作train_step.步骤:每次都随机从训练集中抽取100条样本构成一个mini-batch,并feed给placeholder，然后调用train_step对这些样本进行训练。
for i in range(1000):
  batch_xs, batch_ys = mnist.train.next_batch(100)
  train_step.run({x:batch_xs,y_:batch_ys})

In [0]:
#对模型的准确率进行验证
correct_prediction = tf.equal(tf.argmax(y,1),tf.argmax(y_,1))

In [0]:
#我们统计全部样本预测的accuracy，这里需要先用tf.cast将之前correct_prediction输出的bool值转换为float32，再求平均
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [17]:
print(accuracy.eval({x:mnist.test.images,y_:mnist.test.labels}))

0.9167


通过上面的简单例子，我们使用了TensorFlow实现了一个简单的机器学习算法softmax regression，这可以算作是一个没有隐藏层的最浅的神经网络。整个流程分成4个步骤:
1. 定义算法公式，也就是神经网络forward时的计算
2. 定义loss，选定优化器，并指定优化器优化loss
3. 迭代地对数据进行训练
4. 在测试集或验证集上对准确率进行测评

需要注意的是，TensorFlow和Spark类似，我们定义的各个公式其实只是计算图，在执行这行代码时，计算还没有实际发生，只有等调用run方法，并feed数据时计算才真正执行。

上面我们使用softmax regression达到的准确率为92%，虽然不错，但是**还达不到实用的程度**。手写数字的识别的主要应用场景是识别银行支票，如果准确率不够高，可能会引起严重的后果。后面我们将讲解使用多层感知机和卷积网络，来解决MNIST
手写数字识别问题的方法。